In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
class Node():
    def __init__(self,feature = None ,split_value = None,depth = 0,data=None,Gradients = None, Hessians = None):
        self.feature = feature
        self.split_value = split_value
        self.depth = depth
        self.data = data
        self.left = None
        self.right = None
        self.Gradients = Gradients
        self.Hessians = Hessians 
        self.is_leaf = True if feature ==-1 else False 

    def out(self,lambd,alpha):
        if self.is_leaf == True:
            G = np.sum(self.Gradients)
            H = np.sum(self.Hessians)
    
            if G > alpha:
                return -(G-alpha)/(H+lambd)
            elif G < -alpha:
                return -(G+alpha)/(H+lambd)
            else:
                return 0
    


class Tree():
    def __init__(self,root):
        self.root = root

Basic Structure-

For each boosting round t

1) Start with a guess y[t] (Ive chosen to start with zeroes, we can also start with mean of y_train)
2) Compute the gradients and hessians corresponding to this round for each yi[t] (depends on loss function)
3) Build a tree which maximizes the gain at each step based on features and split points.
4) For finding the feature and split point corresponding to a node we brute force through all possibilities and choose the one that maximizes gain
5) After max_depth is reached in the tree or if the node cannot be split further because gain<0 for all possibilities of feature,split_value then that node becomes a leaf
6) Pass x[i],y_guess[i] from the training set through evaluate_tree. Traverse according to the split point and feature condition at each node
7) Eventually we reach a leaf from where we output -(np.sum(Gradients)/(np.sum(Hessians)+lambda)) where Gradients and Hessians are the arrays corresponding to the data which will come in this leaf after traversing the tree
8) Build max_iter number of rounds and a tree corresponding to each round
9) Pass each x[i] from y_test and traverse the entire tree acc to feature and split_value. Once the sample reaches a leaf add the output of that leaf to the current predictions multiplied by the learning_rate 
10) Do this for all rounds and add all of the outputs up to the initial guess. This is the final prediction

Note -- 
1) It is highly sensitive to learning_rate and initial guess... actual xgboost uses some smoothing or other techniques to decrease sensitivity

2) Based on the structure i thought making a class Round which is a subclass of XGBoost would be clean


In [3]:
class XGBoostRegressor():
    def __init__(self,max_depth = 10,loss = "mse",min_split_loss= 0,learning_rate = 1,l2_regularization = 0,max_iter = 1,l1_regularization = 0,min_child_weight=0,subsample = 1):
        self.max_depth = max_depth
        self.loss = loss.lower()
        self.gamma = min_split_loss
        self.lambd = l2_regularization
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.alpha = l1_regularization
        self.min_child_weight = min_child_weight
        self.subsample = subsample
        self.rounds = []

    
    
    class Round():
        def __init__(self,xgboost_instance,round_num):
            self.xgboost = xgboost_instance
            self.round_num = round_num 
            self.tree = None
            self.Gradients = None
            self.Hessians = None
            self.leaves = []
            self.nodes = []

        def compute_gradients(self,y_train,y_guess):
            if self.xgboost.loss == "mse":
                G = y_guess-y_train
                H = np.ones(y_guess.shape)
            else:
                raise ValueError(f"loss function can only be mse")
            
            self.Gradients = G
            self.Hessians = H
            #print(self.Hessians)

        def find_split_value(self,X,Gradients,Hessians):
            feature =-1
                # if feature isnt found it stays at -1 and node becomes a leaf
            split_value = 0
            gamma = self.xgboost.gamma
            lambd = self.xgboost.lambd
            min_child_weight = self.xgboost.min_child_weight
            G = np.sum(Gradients)
            H = np.sum(Hessians)
            parent_score = 0.5*((G**2)/(H+lambd))
            max_gain = -np.inf
            

            for curr_feature in range(X.shape[1]):  
                sort_mask = X[:,curr_feature].argsort()
                X_sorted = X[sort_mask]
                G_sorted = Gradients[sort_mask]
                H_sorted = Hessians[sort_mask]
                G_L= 0
                H_L = 0
                for i in range(len(X_sorted)-1):
                    curr_split_value = (X_sorted[i, curr_feature] + X_sorted[i+1, curr_feature]) / 2
                    G_L += G_sorted[i]
                    H_L += H_sorted[i]
                    G_R = G - G_L
                    H_R = H - H_L

                    # if equal values come then split point will be wrongly detected
                    if X_sorted[i,curr_feature] == X_sorted[i+1,curr_feature]:
                        continue
                    
                    curr_gain  = 0.5*((G_L**2)/(H_L + lambd) + (G_R**2)/(H_R+lambd)- parent_score) - gamma
                    if (curr_gain >= 0) and (curr_gain > max_gain) :
                        if (H_L >= min_child_weight) and (H_R>=min_child_weight):
                            feature = curr_feature
                            split_value = curr_split_value
                            max_gain = curr_gain
            
            return feature,split_value
        
        def create_node(self,data,depth,Gradients,Hessians):
            feature,split_value = self.find_split_value(data,Gradients=Gradients,Hessians=Hessians)

            
            new_node = Node(feature=feature,split_value=split_value,depth=depth,data=data,Gradients=Gradients,Hessians=Hessians)
            self.nodes.append(new_node)
            return new_node        
        

        def build_tree(self,X):
            
            
            curr_depth = 1
            root = self.create_node(data = X,depth=curr_depth,Gradients=self.Gradients,Hessians=self.Hessians)
            #print(root.is_leaf)
            queue = [root]
            leaves = []
            while((curr_depth+1<=self.xgboost.max_depth) and (len(queue)!=0)):
                curr_node = queue.pop(0)
                curr_depth = curr_node.depth
                curr_data = curr_node.data
                curr_Grads = curr_node.Gradients
                curr_Hess = curr_node.Hessians

                if curr_depth+1 == self.xgboost.max_depth:
                    curr_node.is_leaf = True
                    leaves.append(curr_node)

                if not curr_node.is_leaf:
                    mask_left = curr_data[:,curr_node.feature] < curr_node.split_value
                    mask_right = curr_data[:,curr_node.feature] >= curr_node.split_value


                    # Empty masks is creating NaN values ahead !
                    if (np.sum(mask_left>0) and np.sum(mask_right) > 0) :

                        curr_node.left = self.create_node(depth=curr_depth+1,data = curr_data[mask_left],Gradients= curr_Grads[mask_left],Hessians=curr_Hess[mask_left])
                        curr_node.right = self.create_node(depth=curr_depth+1,data = curr_data[mask_right],Gradients= curr_Grads[mask_right],Hessians=curr_Hess[mask_right])
                        queue.append(curr_node.left)
                        queue.append(curr_node.right)

                    else:
                        curr_node.is_leaf = True
                        leaves.append(curr_node)
                
                elif curr_node.is_leaf:
                    leaves.append(curr_node)
            
            
            self.tree = Tree(root=root)
            self.leaves = leaves


            # for node in self.nodes:
            #     if (node.is_leaf == False) and (node.left == None) and(node.right==None):
            #         print(len(node.data))
        
        def evaluate_tree(self,X,y_guess):
            if self.tree is None or self.tree.root is None:
                raise ValueError("Tree not built yet. Please call fit before predict")
            #print(self.tree.root.left)
            f = []

            for i in range(len(X)):
                curr_node = self.tree.root

                
                while curr_node.is_leaf == False:
                    curr_feature = curr_node.feature
                    curr_split_value = curr_node.split_value

                    if X[i,curr_feature] < curr_split_value:
                        curr_node = curr_node.left
                        
                    else :
                        curr_node = curr_node.right
                    #print(curr_node.is_leaf)
                        
                f.append(curr_node.out(lambd= self.xgboost.lambd,alpha = self.xgboost.alpha))
                #print(curr_node.out(lambd= self.xgboost.lambd,alpha = self.xgboost.alpha))
            f = np.array(f,dtype=float)
            f = f.reshape(y_guess.shape)
            #print(type(f))
            y_guess_new = y_guess + self.xgboost.learning_rate*f
            return y_guess_new
    

    def create_round(self,round_num):
        round = self.Round(xgboost_instance=self,round_num=round_num)
        return round
    
    def fit(self,X_train,y_train):
        if isinstance(X_train,pd.DataFrame):
            X_train = X_train.values
        if (isinstance(y_train,pd.DataFrame)) or (isinstance(y_train,pd.Series)):
            y_train = y_train.values
        y_train = y_train.ravel()
        for i in range(self.max_iter):
            round_num = i+1
            self.rounds.append(self.create_round(round_num))
        
        
        # can use init guess = mean of y_train over here for both training and testing
        y_guess = np.zeros(y_train.shape,dtype = float)
        
        
        for round in self.rounds:
            subsample = self.subsample
            num_select = int(len(X_train)*subsample)
            mask = np.random.choice(len(X_train),size=num_select,replace=False)
            X_train_sub = X_train[mask]
            y_train_sub = y_train[mask]
            y_guess_sub = y_guess[mask]
            round.compute_gradients(y_train_sub,y_guess_sub)
            round.build_tree(X_train_sub)
            y_guess = round.evaluate_tree(X_train,y_guess)
            
        
    
    def predict(self,X_test):
        if isinstance(X_test,pd.DataFrame):
            X_test = X_test.values
        y_guess = np.zeros(X_test.shape[0])


        for round in self.rounds:
            y_guess = round.evaluate_tree(X_test,y_guess)
        return y_guess 


In [4]:
data = pd.read_csv("advertising.csv")
X = data.drop(columns = ["Sales"])
y = data["Sales"]

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8)


In [17]:
manual_XGB = XGBoostRegressor(max_depth=200,learning_rate=1,subsample=0.9,min_child_weight=1,l2_regularization=0.0,l1_regularization=0.001,max_iter=20)
manual_XGB.fit(X_train,y_train)
y_pred = manual_XGB.predict(X_test)
mean_squared_error(y_pred=y_pred,y_true=y_test)


1.0778359999999971

In [18]:
import xgboost as xgb
sklearn_XGB = xgb.XGBRegressor()
sklearn_XGB.fit(X_train,y_train)
y_pred = sklearn_XGB.predict(X_test)
mean_squared_error(y_pred,y_test)

0.9973480584548561